In [1]:
# Imports
import json
import requests
import pandas as pd
import chess.pgn
import io
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import chess.engine
import sys
import logging
import math

# Configure logging to print to stdout
logging.basicConfig(
    level=logging.INFO, format="%(levelname)s: %(message)s", stream=sys.stdout
)

# Configure plotting style
sns.set(style="whitegrid")

# Replace with the actual path to your general population PGN file
# Replace with the actual path to your general population PGN file
GENERAL_PGN_FILE_PATH = "/Users/benjaminrosales/Desktop/Chess Study Materials & Data/Comparison Games/lichess_db_standard_rated_2017-05.pgn"

# Path to your Stockfish executable
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"

# List of ADHD players' usernames (Lichess)
ADHD_USERNAMES = [
    "teoeo",
    "Tobermorey",
    "apostatlet",
    "LovePump1000",
    "Stuntmanandy",
    "Banfy_B",
    "ChessyChesterton12",
    "Yastoon",
    "Timy1976",
    "SonnyDayz11",
    "xiroir",
]

In [2]:
def safe_int(value, default=None):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


def parse_clock_time(comment):
    # Extract clock time from comment, e.g., "%clk 1:23:45.678"
    match = re.search(r"%clk\s+([\d:.]+)", comment)
    if match:
        time_str = match.group(1)
        time_parts = [float(part) for part in time_str.split(":")]
        # Weights for hours, minutes, seconds
        weights = [3600, 60, 1]
        weights = weights[-len(time_parts) :]
        seconds = sum(w * t for w, t in zip(weights, time_parts))
        return seconds
    else:
        # Debug statement to check why clock time is not being parsed
        logging.debug(f"Clock time not found in comment: {comment}")
        return None


def parse_evaluation(comment):
    # Extract evaluation from comment, e.g., "%eval 0.34" or "%eval #5"
    match = re.search(r"%eval\s+([+-]?([0-9]+(\.[0-9]+)?)|#[+-]?[0-9]+)", comment)
    if match:
        eval_str = match.group(1)
        return eval_str
    else:
        return None


def categorize_error(eval_change):
    if eval_change is None:
        return "Unknown"
    if eval_change <= -200:
        return "Blunder"
    elif eval_change <= -100:
        return "Mistake"
    elif eval_change <= -50:
        return "Inaccuracy"
    else:
        return "Normal"


def calculate_material(board):
    # Returns material balance for both sides
    material = {"White": 0, "Black": 0}
    piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 0,  # King is invaluable, but we set to 0 for simplicity
    }
    for piece_type in piece_values:
        value = piece_values[piece_type]
        material["White"] += len(board.pieces(piece_type, chess.WHITE)) * value
        material["Black"] += len(board.pieces(piece_type, chess.BLACK)) * values
    return material


def categorize_game_phase(move_number):
    if move_number <= 15:
        return "Opening"
    elif move_number <= 30:
        return "Middlegame"
    else:
        return "Endgame"

### This is where things need to be more complex - categorizing position_complexity is not enough -- 
def categorize_position_complexity(evaluation):
    if evaluation is None:
        return 'Unknown'
    elif abs(evaluation) < 1:
        return 'Balanced'
    elif abs(evaluation) < 3:
        return 'Slight Advantage'
    else:
        return 'Decisive Advantage'

### - There are code functions in python-chess such as 

def categorize_move(eval_before, eval_after):
    if eval_before is None or eval_after is None:
        return "Unknown"

    # Define saturation limits in centipawns
    SATURATION_LIMIT = 1000  # Equivalent to a 10-pawn advantage
    MATE_SCORE = 10000       # Arbitrary large value representing mate

    # Calculate evaluation change
    eval_change = eval_after - eval_before

    # Handle mate scores (assuming the engine uses large numbers to indicate mate)
    if abs(eval_after) >= MATE_SCORE:
        if eval_after > 0:
            return "Forced Checkmate (Winning)"
        else:
            return "Forced Checkmate (Losing)"

    # Handle evaluation saturation
    if abs(eval_after) >= SATURATION_LIMIT:
        if eval_after > 0:
            return "Winning Position"
        else:
            return "Losing Position"

    # Categorize the move based on evaluation change
    if eval_change <= -300:
        return "Blunder"
    elif eval_change <= -150:
        return "Mistake"
    elif eval_change <= -50:
        return "Inaccuracy"
    elif eval_change >= 300:
        return "Brilliant Move"
    elif eval_change >= 150:
        return "Great Move"
    elif eval_change >= 50:
        return "Good Move"
    else:
        return "Normal"
    
def debug_data_pipeline(df, label):
    # Function definition here
    logging.info(f"Debugging {label}")
    # Process the DataFrame or print logs for debugging

import math  # Ensure math module is imported

def raw_winning_chances(cp):
    MULTIPLIER = -0.00368208
    return 2 / (1 + math.exp(MULTIPLIER * cp)) - 1

def cp_winning_chances(cp):
    cp = max(-1000, min(cp, 1000))
    return raw_winning_chances(cp)

def mate_winning_chances(mate):
    cp = (21 - min(10, abs(mate))) * 100
    signed_cp = cp * (1 if mate > 0 else -1)
    return raw_winning_chances(signed_cp)

def eval_winning_chances(eval_str):
    if eval_str is None:
        return None
    if '#' in eval_str:
        # Mate in N moves
        mate_str = eval_str.replace('#', '')
        try:
            mate = int(mate_str)
            return mate_winning_chances(mate)
        except ValueError:
            return None
    else:
        try:
            cp = float(eval_str) * 100  # Convert from pawns to centipawns
            return cp_winning_chances(cp)
        except ValueError:
            return None



In [3]:
def perform_statistical_test(var, data, test_results, test_type="independent_t"):
    # Prepare data
    group1 = data[data["Group"] == "ADHD"][var].dropna()
    group2 = data[data["Group"] == "General"][var].dropna()

    # Check if data is sufficient
    if len(group1) < 10 or len(group2) < 10:
        logging.warning(f"Not enough data to perform statistical test on '{var}'.")
        return

    # Test for normality
    stat1, p1 = stats.shapiro(group1)
    stat2, p2 = stats.shapiro(group2)
    normal = p1 > 0.05 and p2 > 0.05

    # Test for equal variances
    stat_levene, p_levene = stats.levene(group1, group2)
    equal_var = p_levene > 0.05

    # Choose appropriate test
    if normal and equal_var and test_type == "independent_t":
        # Independent T-test
        stat, p = stats.ttest_ind(group1, group2, equal_var=True)
        test_name = "Independent t-test"
    elif normal and not equal_var and test_type == "independent_t":
        # Welch's T-test
        stat, p = stats.ttest_ind(group1, group2, equal_var=False)
        test_name = "Welch's t-test"
    else:
        # Mann-Whitney U Test
        stat, p = stats.mannwhitneyu(group1, group2, alternative="two-sided")
        test_name = "Mann-Whitney U test"

    test_results.append(
        {"Variable": var, "Test": test_name, "Statistic": stat, "p-value": p}
    )


def perform_chi_squared_test(category_var, data, test_results):
    contingency_table = pd.crosstab(data["Group"], data[category_var])
    if contingency_table.empty or contingency_table.shape[1] == 0:
        logging.warning(f"Contingency table is empty for variable '{category_var}'.")
        return
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    test_results.append(
        {
            "Variable": category_var,
            "Test": "Chi-Squared test",
            "Statistic": chi2,
            "p-value": p,
        }
    )

In [4]:
def fetch_lichess_games(username, max_games=2000):  # Increase max_games
    url = f"https://lichess.org/api/games/user/{username}"
    params = {
        "max": max_games,
        "moves": True,
        "evals": True,  # Include evaluations in the PGN comments
        "clocks": True,  # Include clock times in the PGN comments
    }
    headers = {"Accept": "application/x-chess-pgn"}
    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        logging.warning(
            f"Failed to fetch games for user '{username}'. Status code: {response.status_code}"
        )
        return []
    pgn_text = response.text
    games = []
    pgn_io = io.StringIO(pgn_text)
    while True:
        game = chess.pgn.read_game(pgn_io)
        if game is None:
            break

        # Check if the game contains evaluations
        has_evaluation = False
        node = game
        while node.variations:
            next_node = node.variations[0]
            comment = next_node.comment
            if "%eval" in comment:
                has_evaluation = True
                break
            node = next_node

        if has_evaluation:
            games.append(game)

    logging.info(f"Fetched {len(games)} games with evaluations for user '{username}'.")
    return games


def process_pgn_file(pgn_file_path, max_games=None):
    games = []
    try:
        with open(pgn_file_path, "r", encoding="utf-8") as pgn_file:
            game_counter = 0
            while True:
                game = chess.pgn.read_game(pgn_file)
                if game is None:
                    break

                # Check if the game contains evaluations
                has_evaluation = False
                node = game
                while node.variations:
                    next_node = node.variations[0]
                    comment = next_node.comment
                    if "%eval" in comment:
                        has_evaluation = True
                        break
                    node = next_node

                if has_evaluation:
                    games.append(game)
                    game_counter += 1

                if max_games and game_counter >= max_games:
                    break

        logging.info(
            f"Successfully read {len(games)} games with evaluations from PGN file '{pgn_file_path}'."
        )
    except Exception as e:
        logging.error(f"Failed to read PGN file '{pgn_file_path}': {e}")
    return games


def process_games(games, group_label, engine, max_depth=2):
    all_moves = []
    for game in tqdm(games, desc=f"Processing {group_label} games"):
        try:
            board = game.board()
            game_id = game.headers.get("Site", "Unknown")
            event = game.headers.get("Event", "Unknown")
            date = game.headers.get("UTCDate", "Unknown")
            white = game.headers.get("White", "Unknown")
            black = game.headers.get("Black", "Unknown")
            result = game.headers.get("Result", "Unknown")
            white_elo = safe_int(game.headers.get("WhiteElo", None))
            black_elo = safe_int(game.headers.get("BlackElo", None))
            time_control = game.headers.get("TimeControl", "Unknown")

            node = game
            move_number = 0
            prev_eval = None
            current_material = calculate_material(board)
            prev_time_remaining = None  # Initialize before the loop

            # Check if the game has evaluations
            has_evaluation = False
            temp_node = node
            while temp_node.variations:
                next_temp_node = temp_node.variations[0]
                comment = next_temp_node.comment
                if "%eval" in comment:
                    has_evaluation = True
                    break
                temp_node = next_temp_node

            if not has_evaluation:
                continue  # Skip game if it doesn't have evaluations

            while node.variations:
                next_node = node.variations[0]
                move = next_node.move
                san = board.san(move)
                move_number += 1
                player = "White" if board.turn else "Black"

                # Extract clock time and evaluation from comments
                comment = next_node.comment
                time_remaining = parse_clock_time(comment)
                eval = parse_evaluation(comment)

                # Skip moves without evaluations
                if eval is None:
                    board.push(move)
                    node = next_node
                    prev_time_remaining = time_remaining
                    current_material = calculate_material(board)
                    continue

                # Apply the move to the board
                board.push(move)

                # Calculate time spent
                if time_remaining is not None and prev_time_remaining is not None:
                    time_spent = prev_time_remaining - time_remaining
                    if time_spent < 0:
                        time_spent = None  # Handle clock increments or time resets
                else:
                    time_spent = None

                # Eval change
                if prev_eval is not None and eval is not None:
                    eval_change = eval - prev_eval
                else:
                    eval_change = None

                # Error category
                error_category = categorize_error(eval_change)

                # Material difference after the move
                new_material = calculate_material(board)
                material_diff = new_material[player] - current_material[player]

                # Detect sacrifice
                is_sacrifice = material_diff < 0

                # Categorize game phase
                game_phase = categorize_game_phase(move_number)

                # Categorize position complexity based on previous evaluation
                position_complexity = categorize_position_complexity(prev_eval)

                # Move condition (after move applied)
                move_condition = "Unknown"  # Placeholder

                move_data = {
                    "GameID": game_id,
                    "Event": event,
                    "Date": date,
                    "White": white,
                    "Black": black,
                    "Result": result,
                    "WhiteElo": white_elo,
                    "BlackElo": black_elo,
                    "TimeControl": time_control,
                    "MoveNumber": move_number,
                    "Player": player,
                    "Move": san,
                    "TimeRemaining": time_remaining,
                    "TimeSpent": time_spent,
                    "Evaluation": eval,
                    "EvalChange": eval_change,
                    "WinningChances": winning_chances,
                    "WinningChancesChange": winning_chances_change,
                    "UnderTimePressure": time_remaining is not None
                    and time_remaining < 20,
                    "Group": group_label,
                    "ErrorCategory": error_category,
                    "IsSacrifice": is_sacrifice,
                    "GamePhase": game_phase,
                    "PositionComplexity": position_complexity,
                    "MoveCondition": move_condition,
                }
                all_moves.append(move_data)

                # Update for next iteration
                prev_eval = eval
                prev_time_remaining = time_remaining
                current_material = new_material
                node = next_node
        except Exception as e:
            logging.error(f"Error processing game: {e}")
            continue
    return pd.DataFrame(all_moves)

In [5]:
def process_games(games, group_label, engine, max_depth=2):
    all_moves = []
    for game in tqdm(games, desc=f"Processing {group_label} games"):
        try:
            board = game.board()
            game_id = game.headers.get("Site", "Unknown")
            event = game.headers.get("Event", "Unknown")
            date = game.headers.get("UTCDate", "Unknown")
            white = game.headers.get("White", "Unknown")
            black = game.headers.get("Black", "Unknown")
            result = game.headers.get("Result", "Unknown")
            white_elo = safe_int(game.headers.get("WhiteElo", None))
            black_elo = safe_int(game.headers.get("BlackElo", None))
            time_control = game.headers.get("TimeControl", "Unknown")

            node = game
            move_number = 0
            prev_eval = None
            current_material = calculate_material(board)
            prev_time_remaining = None  # Initialize before the loop
            prev_winning_chances = None  # Track the winning chances from the previous move

            # Check if the game has evaluations
            has_evaluation = False
            temp_node = node
            while temp_node.variations:
                next_temp_node = temp_node.variations[0]
                comment = next_temp_node.comment
                if "%eval" in comment:
                    has_evaluation = True
                    break
                temp_node = next_temp_node

            if not has_evaluation:
                continue  # Skip game if it doesn't have evaluations

            while node.variations:
                next_node = node.variations[0]
                move = next_node.move
                san = board.san(move)
                move_number += 1
                player = "White" if board.turn else "Black"

                # Extract clock time and evaluation from comments
                comment = next_node.comment
                time_remaining = parse_clock_time(comment)
                eval = parse_evaluation(comment)

                # Calculate winning chances
                winning_chances = eval_winning_chances(eval)

                # Calculate winning chances change
                if prev_winning_chances is not None and winning_chances is not None:
                    winning_chances_change = winning_chances - prev_winning_chances
                else:
                    winning_chances_change = None

                # Skip moves without evaluations
                if eval is None:
                    board.push(move)
                    node = next_node
                    prev_time_remaining = time_remaining
                    current_material = calculate_material(board)
                    prev_winning_chances = winning_chances
                    continue

                # Apply the move to the board
                board.push(move)

                # Calculate time spent
                if time_remaining is not None and prev_time_remaining is not None:
                    time_spent = prev_time_remaining - time_remaining
                    if time_spent < 0:
                        time_spent = None  # Handle clock increments or time resets
                else:
                    time_spent = None

                # Eval change
                if prev_eval is not None and eval is not None:
                    eval_change = eval - prev_eval
                else:
                    eval_change = None

                # Error category
                error_category = categorize_error(eval_change)

                # Material difference after the move
                new_material = calculate_material(board)
                material_diff = new_material[player] - current_material[player]

                # Detect sacrifice
                is_sacrifice = material_diff < 0

                # Categorize game phase
                game_phase = categorize_game_phase(move_number)

                # Categorize position complexity based on previous evaluation
                position_complexity = categorize_position_complexity(prev_eval)

                # Move condition (after move applied)
                move_condition = "Unknown"  # Placeholder

                move_data = {
                    "GameID": game_id,
                    "Event": event,
                    "Date": date,
                    "White": white,
                    "Black": black,
                    "Result": result,
                    "WhiteElo": white_elo,
                    "BlackElo": black_elo,
                    "TimeControl": time_control,
                    "MoveNumber": move_number,
                    "Player": player,
                    "Move": san,
                    "TimeRemaining": time_remaining,
                    "TimeSpent": time_spent,
                    "Evaluation": eval,
                    "EvalChange": eval_change,
                    "WinningChances": winning_chances,
                    "WinningChancesChange": winning_chances_change,
                    "UnderTimePressure": time_remaining is not None and time_remaining < 20,
                    "Group": group_label,
                    "ErrorCategory": error_category,
                    "IsSacrifice": is_sacrifice,
                    "GamePhase": game_phase,
                    "PositionComplexity": position_complexity,
                    "MoveCondition": move_condition,
                }
                all_moves.append(move_data)

                # Update for next iteration
                prev_eval = eval
                prev_time_remaining = time_remaining
                current_material = new_material
                prev_winning_chances = winning_chances
                node = next_node
        except Exception as e:
            logging.error(f"Error processing game: {e}")
            continue
    return pd.DataFrame(all_moves)

In [6]:
# ----------------------- 1. Fetch and Process ADHD Players' Games -----------------------

adhd_games = []
for username in ADHD_USERNAMES:
    logging.info(f"Fetching games for user '{username}'...")
    user_games = fetch_lichess_games(username, max_games=2000)  # Adjust max_games as needed
    adhd_games.extend(user_games)

if not adhd_games:
    logging.warning("No ADHD games fetched. Exiting analysis.")
else:
    # Initialize the chess engine
    try:
        engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
        logging.info(f"Initialized Stockfish engine at '{STOCKFISH_PATH}'.")
    except FileNotFoundError:
        logging.critical(f"Stockfish executable not found at '{STOCKFISH_PATH}'. Please update the path.")
        engine = None
    except Exception as e:
        logging.critical(f"Failed to initialize Stockfish engine: {e}")
        engine = None

    if engine is not None:
        # ----------------------- 2. Process ADHD Players' Games -----------------------
        
        logging.info("Processing ADHD players' games...")
        adhd_moves_df = process_games(adhd_games, group_label='ADHD', engine=engine)
        debug_data_pipeline(adhd_moves_df, "ADHD GAMES PROCESSING")
        
        # ----------------------- 3. Fetch and Process General Population Games -----------------------
        
        logging.info("Fetching general population games...")
        if not os.path.exists(GENERAL_PGN_FILE_PATH):
            logging.error(f"PGN file not found at path: {GENERAL_PGN_FILE_PATH}")
            general_games = []
        else:
            general_games = process_pgn_file(GENERAL_PGN_FILE_PATH, max_games=2000)  # Adjust max_games as needed
        
        if not general_games:
            logging.warning("No General population games to process.")
            general_moves_df = pd.DataFrame()
        else:
            logging.info("Processing general population games...")
            general_moves_df = process_games(general_games, group_label='General', engine=engine)
            debug_data_pipeline(general_moves_df, "GENERAL GAMES PROCESSING")
        
        # ----------------------- 4. Combine Datasets -----------------------

        logging.info("Combining datasets...")
        all_moves_df = pd.concat([adhd_moves_df, general_moves_df], ignore_index=True)
        debug_data_pipeline(all_moves_df, "COMBINED DATASET")

        # ----------------------- 5. Data Cleaning -----------------------

        logging.info("Cleaning data...")
        required_columns = ['TimeSpent', 'Evaluation', 'EvalChange', 'WhiteElo', 'BlackElo']
        # Since we've filtered out moves without evaluations, we can expect 'Evaluation' and 'EvalChange' to be present
        all_moves_df = all_moves_df.dropna(subset=required_columns)

        # Ensure 'IsSacrifice' is boolean
        all_moves_df['IsSacrifice'] = all_moves_df['IsSacrifice'].fillna(False).astype(bool)

        # Convert relevant columns to numeric types
        numeric_columns = ['TimeSpent', 'Evaluation', 'EvalChange', 'WhiteElo', 'BlackElo']
        for col in numeric_columns:
            all_moves_df[col] = pd.to_numeric(all_moves_df[col], errors='coerce')

        # Drop rows with NaNs resulted from non-numeric conversion
        all_moves_df = all_moves_df.dropna(subset=numeric_columns)

        # After cleaning, output the number of moves remaining
        logging.info(f"Total number of moves after cleaning: {len(all_moves_df)}")

        # ----------------------- 6. Statistical Testing -----------------------
        
        logging.info("Performing statistical tests...")
        test_results = []
        
        # ----------------------- 7. Analysis and Plotting -----------------------

        #these functions currently make no sense, and I need to work on plots - what actually I want displayed
    
        
        logging.info("Generating plots and performing statistical tests...")

INFO: Fetching games for user 'teoeo'...
INFO: Fetched 981 games with evaluations for user 'teoeo'.
INFO: Fetching games for user 'Tobermorey'...
INFO: Fetched 75 games with evaluations for user 'Tobermorey'.
INFO: Fetching games for user 'apostatlet'...
INFO: Fetched 219 games with evaluations for user 'apostatlet'.
INFO: Fetching games for user 'LovePump1000'...
INFO: Fetched 391 games with evaluations for user 'LovePump1000'.
INFO: Fetching games for user 'Stuntmanandy'...
INFO: Fetched 816 games with evaluations for user 'Stuntmanandy'.
INFO: Fetching games for user 'Banfy_B'...
INFO: Fetching games for user 'ChessyChesterton12'...
INFO: Fetched 228 games with evaluations for user 'ChessyChesterton12'.
INFO: Fetching games for user 'Yastoon'...
INFO: Fetched 24 games with evaluations for user 'Yastoon'.
INFO: Fetching games for user 'Timy1976'...
INFO: Fetching games for user 'SonnyDayz11'...
INFO: Fetched 16 games with evaluations for user 'SonnyDayz11'.
INFO: Fetching games for u

Processing ADHD games: 100%|██████████| 2854/2854 [00:03<00:00, 796.25it/s]


INFO: Debugging ADHD GAMES PROCESSING
INFO: Fetching general population games...
INFO: Successfully read 2000 games with evaluations from PGN file '/Users/benjaminrosales/Desktop/Chess Study Materials & Data/Comparison Games/lichess_db_standard_rated_2017-05.pgn'.
INFO: Processing general population games...


Processing General games: 100%|██████████| 2000/2000 [00:02<00:00, 820.93it/s]


INFO: Debugging GENERAL GAMES PROCESSING
INFO: Combining datasets...
INFO: Debugging COMBINED DATASET
INFO: Cleaning data...
INFO: Total number of moves after cleaning: 149923
INFO: Performing statistical tests...
INFO: Generating plots and performing statistical tests...


In [ ]:
#%%
import matplotlib.pyplot as plt
import numpy as np

def plot_performance_under_time_pressure(data, test_results):
    """
    Plot the performance of players under time pressure.
    """
    time_pressure_data = data[data['UnderTimePressure'] == True]
    plt.figure(figsize=(10, 5))
    plt.hist(time_pressure_data['EvalChange'], bins=30, alpha=0.7, label="Evaluation Change")
    plt.xlabel("Evaluation Change")
    plt.ylabel("Frequency")
    plt.title("Performance under Time Pressure")
    plt.legend()
    plt.show()
    # Example test result (add more meaningful stats as needed)
    test_results.append({'Variable': 'Performance Under Time Pressure', 'Test': 'Example Test', 'Statistic': 1.23, 'p-value': 0.04})

def plot_accuracy_vs_time(data, test_results):
    """
    Plot accuracy of moves versus time taken.
    """
    plt.figure(figsize=(10, 5))
    plt.scatter(data['TimeSpent'], data['EvalChange'], alpha=0.5, label="Move Accuracy vs Time")
    plt.xlabel("Time Spent on Move (s)")
    plt.ylabel("Evaluation Change")
    plt.title("Move Accuracy vs Time Spent")
    plt.legend()
    plt.show()
    # Placeholder test result
    test_results.append({'Variable': 'Accuracy vs Time', 'Test': 'Correlation', 'Statistic': 0.45, 'p-value': 0.03})

def plot_error_rate(data, test_results):
    """
    Plot error rate over different phases of the game.
    """
    error_data = data[data['ErrorCategory'] != 'Normal']
    plt.figure(figsize=(10, 5))
    plt.hist(error_data['GamePhase'], alpha=0.7, label="Error Rate by Game Phase")
    plt.xlabel("Game Phase")
    plt.ylabel("Frequency of Errors")
    plt.title("Error Rate across Game Phases")
    plt.legend()
    plt.show()
    # Placeholder test result
    test_results.append({'Variable': 'Error Rate', 'Test': 'Frequency Analysis', 'Statistic': 2.76, 'p-value': 0.02})

def plot_time_management(data, test_results):
    """
    Plot time management patterns for players.
    """
    plt.figure(figsize=(10, 5))
    plt.hist(data['TimeRemaining'].dropna(), bins=50, alpha=0.7, label="Time Remaining per Move")
    plt.xlabel("Time Remaining (s)")
    plt.ylabel("Move Frequency")
    plt.title("Time Management Patterns")
    plt.legend()
    plt.show()
    # Placeholder test result
    test_results.append({'Variable': 'Time Management', 'Test': 'Distribution Analysis', 'Statistic': 3.14, 'p-value': 0.01})

def stratify_by_elo(data, test_results):
    """
    Stratify and analyze data by Elo rating.
    """
    plt.figure(figsize=(10, 5))
    plt.hist(data['WhiteElo'].dropna(), bins=50, alpha=0.7, label="White Elo Distribution")
    plt.hist(data['BlackElo'].dropna(), bins=50, alpha=0.7, label="Black Elo Distribution", color='orange')
    plt.xlabel("Elo Rating")
    plt.ylabel("Frequency")
    plt.title("Elo Distribution among Players")
    plt.legend()
    plt.show()
    # Placeholder test result
    test_results.append({'Variable': 'Elo Stratification', 'Test': 'Elo Distribution', 'Statistic': 4.56, 'p-value': 0.05})

# These functions now generate basic plots for each respective analysis and add placeholder test results.
# This will allow the rest of the notebook to execute without errors.

        
        # ----------------------- 8. Display Statistical Test Results -----------------------
        
        logging.info("\n----------------------- Statistical Test Results -----------------------\n")
        results_df = pd.DataFrame(test_results)
        
        if not results_df.empty:
            # Apply Bonferroni correction for multiple comparisons
            num_tests = len(results_df)
            results_df['Adjusted p-value'] = results_df['p-value'] * num_tests
            results_df['Adjusted p-value'] = results_df['Adjusted p-value'].apply(lambda x: min(x, 1.0))
            
            # Determine significance after correction
            results_df['Significant'] = results_df['Adjusted p-value'] < 0.05
            
            # Display the results
            display(results_df[['Variable', 'Test', 'Statistic', 'p-value', 'Adjusted p-value', 'Significant']])
            
            logging.info("\nNote: p-values have been adjusted using the Bonferroni correction for multiple comparisons.\n")
        else:
            logging.info("No statistical tests were performed.")
        
        # ----------------------- Cleanup -----------------------
        
        # Close the chess engine
        engine.quit()
        
        logging.info("Analysis complete.")

IndentationError: unexpected indent (2456860323.py, line 84)